In [1]:
from skimage import io
import numpy as np
import matplotlib.image as mimg
import cv2
from skimage.color import rgb2gray

In [2]:
def invertedBinaryImage(image):
    img = image.copy()
    shape = np.shape(img)
    if len(shape) > 2:
        if shape[2] > 3:
            img = rgb2gray(img[:,:,0:3])*255
        else:
            img = rgb2gray(img)*255
    b_img = np.zeros((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] != 255:
                b_img[v][u] = 1
    return b_img

In [3]:
def discreteContrast(filteredImage):
    shape = np.shape(filteredImage)
    img = filteredImage.copy()
    valueList = np.unique(img)
    valueDict = dict()
    for i in range(0, len(valueList)):
        valueDict.update({valueList[i] : i})
    for v in range(shape[0]):
        for u in range(shape[1]):
            img[v][u] = valueDict.get(filteredImage[v][u])
    return img

In [4]:
def invBiImg(img):
    ret_img = img.copy()
    shape = np.shape(img)
    b_img = np.ones((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] == 0:
                pass

In [5]:
def labelForeGround(image):
    img = rgb2gray(image)
    shape = np.shape(img)
    b_img = np.zeros((shape[0], shape[1]))
    for v in range(0, shape[0]):
        for u in range(0, shape[1]):
            if img[v][u] != 0:
                b_img[v][u] = 1
    return b_img

In [6]:
def invert(image):
    img = image.copy()
    img = (img * -1) + np.amax(image)
    return img

In [7]:
def labeledNeighbors(image, x, y, n=8):
    nbrs = list()
    if x - 1 >= 0 and image[y][x-1] > 1:
        nbrs.append((x-1, y))
    if n == 4:
        if y - 1 >= 0 and image[y-1][x] > 1:
            nbrs.append((x, y-1))
        return nbrs
    r = -1
    v = 2
    if x - 1 < 0:
        r = 0
    if x + 1 >= np.shape(image)[1]:
        v = 1
    if y - 1 >= 0:
        for i in range(r, v):
            if image[y-1][x+i] > 1:
                nbrs.append((x+i, y-1))
    return nbrs

In [13]:
def sequentialLabeling(image, n=8):
    # img = invertedBinaryImage(image)
    img = image.copy()
    m = 2
    c = list()
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] == 1:
                nbrs = labeledNeighbors(img, u, v, n)
                if len(nbrs) == 0:
                    img[v][u] = m
                    m = m + 1
                elif len(nbrs) == 1:
                    p = nbrs[0]
                    img[v][u] = img[p[1]][p[0]]
                elif len(nbrs) > 1:
                    p = nbrs.pop(0)
                    k = img[p[1]][p[0]]
                    img[v][u] = k
                    for p in nbrs:
                        ni = img[p[1]][p[0]]
                        if ni != k:
                            c.append({ni, k})
    
    r = list()
    for i in range(2, m):
        r.append({i})
    
    for collisions in c:
        a = collisions.pop()
        b = collisions.pop()
        for labelsets in r:
            if labelsets.issuperset({a}):
                ra = labelsets
            if labelsets.issuperset({b}):
                rb = labelsets
        if ra.isdisjoint(rb):
            ra.update(rb)
            r.remove(rb)
    
    for v in range(0, np.shape(img)[0]):
        for u in range(0, np.shape(img)[1]):
            if img[v][u] > 1:
                for lsets in r:
                    if lsets.issuperset({img[v][u]}):
                        img[v][u] = min(lsets)
    return invert(img)

In [9]:
def countAreaSize(sql_img):
    valueList, count = np.unique(sql_img, return_counts=True)
    return valueList[0:len(valueList)-1], count[0:len(count)-1]

In [16]:
def process(filename):
    img = io.imread(filename)
    bimg = labelForeGround(img)
    sql = discreteContrast(sequentialLabeling(bimg))
    labels, counts = countAreaSize(sql)
    return removeClutter(counts)

In [11]:
def removeClutter(array, threshold=1000):
    ret = []
    for i in array:
        if i > threshold:
            ret.append(i)
    return ret

In [14]:
img = io.imread('0.png')
bimg = labelForeGround(img)
sql = discreteContrast(sequentialLabeling(bimg))
io.imsave('binarytest.png', bimg)
io.imsave('labeledtest.png', sql)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.0]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
img = io.imread('highcontrast/1Euro0.png')
bimg = labelForeGround(img)
sql = discreteContrast(sequentialLabeling(bimg))
io.imsave('binarytest.png', bimg)
io.imsave('labeledtest.png', sql)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 3.0]. Convert image to uint8 prior to saving to suppress this warning.


In [ ]:
print(countAreaSize(sql))

(array([0., 1., 2.]), array([4182,    1,    5], dtype=int64))


In [18]:
coinDict = dict()
mi  = 10000
mx = 0
names = ['2Euro', '1Euro', '20Cent', '10Cent', '5Cent']
for n in names:
    mi  = 10000
    mx = 0
    for i in range(0,5):
        filename = f'highcontrast/{n}{i}.png'
        pi = process(filename)
        for p in pi:
            if p <= mi:
                mi = p
            if p >= mx:
                mx = p
    coinDict.update({n : (mi, mx)})
for c in coinDict:
    print(c)
    print(coinDict.get(c))
    

2Euro
(4462, 5089)
1Euro
(3758, 4228)
20Cent
(3399, 3883)
10Cent
(2698, 3041)
5Cent
(3254, 3601)


In [19]:
names = ['2Euro', '1Euro', '20Cent', '10Cent', '5Cent']
for n in names:
    for i in range(0,5):
        filename = f'highcontrast/{n}{i}.png'
        print(filename)
        print(process(filename))
    

highcontrast/2Euro0.png
[4953]
highcontrast/2Euro1.png
[5089]
highcontrast/2Euro2.png
[4856]
highcontrast/2Euro3.png
[4462]
highcontrast/2Euro4.png
[4630]
highcontrast/1Euro0.png
[4182]
highcontrast/1Euro1.png
[4228]
highcontrast/1Euro2.png
[4113]
highcontrast/1Euro3.png
[3758]
highcontrast/1Euro4.png
[3902]
highcontrast/20Cent0.png
[3883]
highcontrast/20Cent1.png
[3873]
highcontrast/20Cent2.png
[3740]
highcontrast/20Cent3.png
[3399]
highcontrast/20Cent4.png
[3505]
highcontrast/10Cent0.png
[3037]
highcontrast/10Cent1.png
[3041]
highcontrast/10Cent2.png
[2976]
highcontrast/10Cent3.png
[2698]
highcontrast/10Cent4.png
[2795]
highcontrast/5Cent0.png
[3578]
highcontrast/5Cent1.png
[3601]
highcontrast/5Cent2.png
[3485]
highcontrast/5Cent3.png
[3254]
highcontrast/5Cent4.png
[3379]
